# Calysto Scheme Tensorflow Example

First we install the Python modules we will need:

In [68]:
! pip install comet_ml tensorflow numpy --upgrade --quiet

And import them:

In [2]:
(import-as "tensorflow" "tf")
(import-as "numpy" "np")
(import "comet_ml")
(import "PIL.Image")
(import "base64")
(import "IPython")

(IPython)

To use Comet to track your experiments, copy your API key from:
https://www.comet.com/account-settings/apiKeys

and paste it below between the quotes. (Otherwise, skip this cell).

In [3]:
(import "os")
(set-item! os.environ "COMET_API_KEY" "<PASTE-COMET-API-KEY-HERE>")

Next, we get the MNIST dataset:

In [4]:
(define mnist tf.keras.datasets.mnist)

In [5]:
(define dataset (mnist.load_data))

In [6]:
(define x_train (get-item (get-item dataset 0) 0))
(define y_train (get-item (get-item dataset 0) 1))
(define x_test (get-item (get-item dataset 1) 0))
(define y_test (get-item (get-item dataset 1) 1))

*Hack to allow Python 3.9 to work:*

In [7]:
(set-attr! base64 "encodestring" base64.encodebytes)

Let's take a look at an input matrix:

In [24]:
(PIL.Image.fromarray (get-item x_train 0))

What is the target for the above?

In [39]:
(get-item y_train 0)

5

## Prepare the dataset

For this network, we'll scale the inputs to be between 0 and 1:

In [27]:
(set! x_train (/ x_train 255.0))

In [28]:
(set! x_test (/ x_test 255.0))

## Customize the Loss function

In [29]:
(define loss_fn (tf.keras.losses.SparseCategoricalCrossentropy (dict '((from_logits : #t)))))

Ok, let's train a model!

In [36]:
(let* ((optimizer (choose "adam" "rmsprop" "sgd"))
       (dropout_rate (choose 0.0 0.1 0.2 0.4))
       (activation (choose "relu" "sigmoid"))
       (hidden_layer_size (choose 10 20 30))
       (options (dict `((optimizer : ,optimizer)(loss : ,loss_fn)(metrics : ,(vector "accuracy")))))
       (epochs 5)
       (experiment (comet_ml.Experiment (dict '((project_name : "calysto-scheme")))))
       (model (tf.keras.models.Sequential 
          (vector
            (tf.keras.layers.Flatten (dict '((input_shape : (28 28)))))
            (tf.keras.layers.Dense hidden_layer_size (dict `((activation : ,activation))))
            (tf.keras.layers.Dropout dropout_rate)
            (tf.keras.layers.Dense 10)
        ))))
  (print experiment.url)
  (model.compile options)
  (model.summary)
  (experiment.log_parameters (dict `((optimizer : ,optimizer) 
                                     (dropout_rate : ,dropout_rate) 
                                     (activation : ,activation) 
                                     (hidden_layer_size : ,hidden_layer_size)
                                     (epochs : ,epochs)
                                    )) (dict))
  (experiment.set_model_graph model)
  (let ((history (model.fit x_train y_train (dict `((epochs : ,epochs)))))
        (step 0))
      (map (lambda (key)
          (set! step 0)
          (map (lambda (v) (experiment.log_metric key v step) (set! step (+ step 1)))
               (get-item history.history key)))
       history.history))
  (experiment.end)
)

"https://www.comet.com/dsblank/calysto-scheme/5ab05a53032f4438a5aa27c97f461bbf"
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_4 (Flatten)         (None, 784)               0         
                                                                 
 dense_8 (Dense)             (None, 10)                7850      
                                                                 
 dropout_4 (Dropout)         (None, 10)                0         
                                                                 
 dense_9 (Dense)             (None, 10)                110       
                                                                 
Total params: 7,960
Trainable params: 7,960
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.5386 - accuracy: 0.8441
Epo

Nice! Now let's do it again, picking different hyperparameters:

In [37]:
(choose)

"https://www.comet.com/dsblank/calysto-scheme/dc1f0e7f33844b89827218c9d099d78d"
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_5 (Flatten)         (None, 784)               0         
                                                                 
 dense_10 (Dense)            (None, 20)                15700     
                                                                 
 dropout_5 (Dropout)         (None, 20)                0         
                                                                 
 dense_11 (Dense)            (None, 10)                210       
                                                                 
Total params: 15,910
Trainable params: 15,910
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3972 - accuracy: 0.8883
E

Continue until there are no more options.